# CS229 Problem Set4
## Problem 6 强化学习：倒立摆

`cart_pole.py`为我们所使用的仿真器，可以用于模仿动作的进行，返回cart的状态，展示当前的cart等。
`control.py` 为我们所需要进行决策的代码，其中给出了一定的框架，我们仅需要填写部分标明的了核心代码。

**首先让我们导入所需要的库文件**

In [1]:
from __future__ import division, print_function  # 兼容性
from cart_pole import CartPole, Physics
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import lfilter

## 初始化
### 仿真器变量
为框架提供的代码，不需要自己填写，但可以根据自己的需求更改

In [2]:
pause_time = 0.0001
min_trial_length_to_start_display = 100
display_started = min_trial_length_to_start_display == 0

NUM_STATES = 163
NUM_ACTIONS = 2
GAMMA = 0.995
TOLERANCE = 0.01
NO_LEARNING_THRESHOLD = 20

- `pause_time` :控制展示时帧与帧之间的时间，值越大，播放的速度就越慢
- `min_trial_length_to_start_display` :只有在成功平衡这么多次的实验后，才允许进行展示。设置其为一个合理的高值（约为100）可以允许你快死的进行学习，并在有了一定的性能之后进行展示。
- `display_started` ： 控制是否开始展示，为1时进行展示

- `NUM_STATES` ： 总共的离散状态数。所有的状态从0到`NUM_STAES-1`进行编号，最后一个状态`NUM_STAES-1`是一个特殊状态，表示着实验失败（pole倒下或cart出界）
- `NUM_ACTIONS` : 仅有2个动作，0代表向右推，1代表向左拉
- `GAMMA` ： 折扣因子`\gamma`
- `TOLERANCE` : 控制每一轮价值迭代的收敛标准，小于这个值代表收敛。
- `NO_LEARNING_THRESHOLD` ：当我们有`NO_LEARNING_THRESHOLD`个连续的价值函数的计算都收敛了，我们可以假设我们的所有的计算都收敛了。

### 其他变量

In [3]:
time = 0 # 循环的次数

time_steps_to_failure = []  #记录第几次失败
num_failures = 0            #记录失败了多少次
time_at_start_of_current_trial = 0 #进行当前实验时是第几次

max_failures = 500          # 在失败这么多次前还没收敛，就停止

cart_pole = CartPole(Physics()) #初始化小车和极点

x, x_dot, theta, theta_dot = 0.0, 0.0, 0.0, 0.0  #连续状态向量
state_tuple = (x, x_dot, theta, theta_dot)
state = cart_pole.get_state(state_tuple) # 离散的状态


### 填写初始化代码
**要求：** 
1. 假设还没有观察到状态转移和反馈
2. 随机初始化价值函数向量为小的随机数（0到0.10）
3. 均匀地初始化状态转移概率
4. 初始化所有反馈为0

In [5]:
###### BEGIN YOUR CODE ######
# TODO:
value = np.random.random((NUM_STATES,1))* 0.1
trans_probs = np.ones((NUM_STATES,NUM_STATES,NUM_ACTIONS))/NUM_STATES
trans_counts = np.zeros((NUM_STATES,NUM_STATES,NUM_ACTIONS))
reward_counts = np.zeros((NUM_STATES,NUM_ACTIONS))
reward = np.zeros((NUM_STATES,1))

#raise NotImplementedError('Initalizations not implemented')
###### END YOUR CODE ######

### 填写MDP模型训练代码
**要求：**
1. 随机选择动作（0或者1）
2. 根据当前的价值函数选择最优策略
即$\pi^{*}(s)=\arg \max _{a \in A} \sum_{s^{\prime} \in S} P_{s a}\left(s^{\prime}\right) V^{*}\left(s^{\prime}\right)$

In [ ]:
consecutive_no_learning_trials = 0
while consecutive_no_learning_trials < NO_LEARNING_THRESHOLD:
       ###### BEGIN YOUR CODE ######
    # TODO:
    score1 = trans_probs[state,:,0].dot(value)  ## 计算当前状态的价值函数
    score2 = trans_probs[state,:,1].dot(value)

    if score1>score2:
        action = 0
    elif score1<score2:
      action = 1
    else:
        action = 0 if np.random.uniform() < 0.5 else 1
    # raise NotImplementedError('Action choice not implemented')
    # action = 0 if np.random.uniform() < 0.5 else 1
    ###### END YOUR CODE ######


In [ ]:
    state_tuple = cart_pole.simulate(action, state_tuple)
    # x, x_dot, theta, theta_dot = state_tuple

    # 随着每次的模拟而递增
    time = time + 1

    # 得到新的离散化的状态
    new_state = cart_pole.get_state(state_tuple)
    # if display_started == 1:
    #     cart_pole.show_cart(state_tuple, pause_time)

    # reward function to use - do not change this!
    if new_state == NUM_STATES - 1:
        R = -1
    else:
        R = 0
        #R = -np.abs(state_tuple[theta])/2.0

### 填写模型更新代码
我们已经得到了从`state`到`new_state`使用的`action`，和新状态下的反馈
**要求**：
1. 记录`state,action,new_state`产生次数的编号
2. 记录每一个`new_state`的反馈
3. 记录达到`new_state`的次数

In [ ]:
     ###### BEGIN YOUR CODE ######
    # TODO:
    trans_counts[state,new_state,action] += 1
    reward_counts[new_state,0] += R
    reward_counts[new_state,1] += 1
    #raise NotImplementedError('Update T and R not implemented')
    ###### END YOUR CODE ######

4.更新MDP的转移概率和反馈

In [ ]:
    if new_state == NUM_STATES - 1:
        for a in range(NUM_ACTIONS):
            for s in range(NUM_STATES):
                den = np.sum(trans_counts[s,:,a])
                if den>0:
                    trans_probs[s,:,a] = trans_counts[s,:,a] / den
        for s in range(NUM_STATES):
            if reward_counts[s,1]>0:
                reward[s] = reward_counts[s,0] / reward_counts[s,1]
        #raise NotImplementedError('MDP  T and R update not implemented')

### 填写价值迭代算法
**要求：**
1. 使用参数`TOLERANCE`来判断收敛
2. 若在第一轮迭代中收敛，更新变量，并检测整个程序是否该截止

In [ ]:
     ###### BEGIN YOUR CODE ######
        # TODO:
        iterations = 0
        while True:
            iterations += 1
            new_value = np.zeros((NUM_STATES, 1))  #必须在内层循环进行初始化，否则将会导致value和new_value公用同一内存地址
            for s in range(NUM_STATES):
                t_values = []
                for a in range(NUM_ACTIONS) :
                    t_values.append(trans_probs[s,:,a].dot(value))
                new_value[s] = np.max(t_values)
            new_value = reward + GAMMA*new_value
            diff = np.max(np.abs(value-new_value))
            value = new_value
            if diff<TOLERANCE:
                break
        if iterations==1:
            consecutive_no_learning_trials +=1
        else:
            consecutive_no_learning_trials =0

        #raise NotImplementedError('Value iteration choice not implemented')
        ###### END YOUR CODE ######

### 剩余框架代码

In [ ]:
    # when the pole fell and the state must be reinitialized.
    if new_state == NUM_STATES - 1:
        num_failures += 1
        if num_failures >= max_failures:
            break
        print('[INFO] Failure number {}'.format(num_failures))
        time_steps_to_failure.append(time - time_at_start_of_current_trial)
        # time_steps_to_failure[num_failures] = time - time_at_start_of_current_trial
        time_at_start_of_current_trial = time

        if time_steps_to_failure[num_failures - 1] > min_trial_length_to_start_display:
            display_started = 1

        # Reinitialize state
        # x = 0.0
        x = -1.1 + np.random.uniform() * 2.2
        x_dot, theta, theta_dot = 0.0, 0.0, 0.0
        state_tuple = (x, x_dot, theta, theta_dot)
        state = cart_pole.get_state(state_tuple)
    else:
        state = new_state

# plot the learning curve (time balanced vs. trial)
log_tstf = np.log(np.array(time_steps_to_failure))
plt.plot(np.arange(len(time_steps_to_failure)), log_tstf, 'k')
window = 30
w = np.array([1/window for _ in range(window)])
weights = lfilter(w, 1, log_tstf)
x = np.arange(window//2, len(log_tstf) - window//2)
plt.plot(x, weights[window:len(log_tstf)], 'r--')
plt.xlabel('Num failures')
plt.ylabel('Num steps to failure')
plt.show()
